# Nonlinear MHD code in one dimension
Normalized (ideal) MHD equations for denstiy $\rho$, velocity $\textbf{u}$, pressure $p$ and magnetic field $\textbf{B}$:



\begin{align}
\partial_t\rho+\partial_x\left(\rho u_x\right)=0 \\
\partial_t\left(\rho\textbf{u}\right)+\left(\textbf{e}_x\cdot\partial_x\right)\cdot\left[\rho\textbf{uu}+\frac{1}{2}\left(p+B^2\right)-\textbf{BB}\right]=0 \\
\partial_t\textbf{B}=\left(\textbf{e}_x\cdot\partial_x\right)\times\left(\textbf{u}\times\textbf{B}\right)+\eta\partial_x^2\textbf{B} \\
\partial_t p=\partial_x\left(pu_x\right)-(\gamma-1)p\partial_x u_x
\end{align}






This system can be written as a conservation law $\partial_t\textbf{U}+\partial_x\textbf{F}(\textbf{U})=0$ or $\partial_t\textbf{U}+\partial_x\textbf{F}(\textbf{U})=\partial_x^2\textbf{D}(\textbf{U})$ respectively, if $\eta\neq 0$ with $\textbf{U}=\left(\rho,\rho u_x,\rho u_y,\rho u_z,B_x,B_y,B_z,\epsilon\right)^T$, where the energy density is given by
\begin{align}
\epsilon=\frac{1}{2}\rho u^2+\frac{p}{2(\gamma-1)}+\frac{B^2}{2}.
\end{align}
Two Finite Differences methods are implemented: 1. A Lax-Wendroff method for the discretization of the advective part combined with a FTCS (Forward time centered space) scheme for the diffusive part. 2. A Leapfrog method for the discretization of the advective part combined with a DuFort-Frankel scheme for the diffusive part. To smoothen unphysical grid oscillations (especially for the second scheme) artificial viscosity of the form $\nu\partial_x^2\textbf{U}$ is added to every equation exept for the equation for the magnetic field. 

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from JSAnimation import IPython_display
from matplotlib import animation
from scipy.linalg import block_diag
from numpy.linalg import inv
from numpy import linalg as LA
from sympy import *



# Author: Florian Holderied
# Last update: 15.03.2018


# Input Parameters
rho0 = 1       # Equilibrium mass density of plasma
p0 = 0.5       # Equilibrium pressure of plasma
B0x = 1        # Equilibrium magnetic field x
B0y = 0        # Equilibrium magnetic field y
B0z = 0        # Equilibrium magnetic field z
gamma = 5/3    # Adiabatic exponent
eta = 0.1      # Resistivity
nu = 0.00      # Artificial Viscosity
a = 0.01       # Amplitude of largest pertubation from equilibrium
L = 40         # Length of x-domain in + and - direction (L_tot=2L)
T = 200        # Simulation time
Nx = 800       # Number of grid cells in x-direction
CFL = 1        # CFL-condition (determines time step, must be smaller or equal to 1)
init = 1       # Shape of initial condition (1: sine shaped, 2: 1/cosh shaped)
method = 0     # Integration scheme (0: Lax-Wedroff-FTCS, 1: Leapfrog-DuFort-Frankel)
fr = 10        # Plot only every "fr-th" frame in animation







# Linearized advection matrix with order (u1x,u1y,u1z,B1x,B1y,B1z,rho1,p1)
A0 = np.array([0,0,0,0,B0y/rho0,B0z/rho0,0,1/(2*rho0)])
A1 = np.array([0,0,0,0,-B0x/rho0,0,0,0])                
A2 = np.array([0,0,0,0,0,-B0x/rho0,0,0])                
A3 = np.array([0,0,0,0,0,0,0,0])                
A4 = np.array([B0y,-B0x,0,0,0,0,0,0])                
A5 = np.array([B0z,0,-B0x,0,0,0,0,0])                
A6 = np.array([rho0,0,0,0,0,0,0,0])                
A7 = np.array([p0*gamma,0,0,0,0,0,0,0])              
A = np.array([A0,A1,A2,A3,A4,A5,A6,A7])


# Linearized diffusion matrix with order (u1x,u1y,u1z,B1x,B1y,B1z,rho1,p1)
D0 = np.array([nu,0,0,0,0,0,0,0])
D1 = np.array([0,nu,0,0,0,0,0,0])                
D2 = np.array([0,0,nu,0,0,0,0,0])                
D3 = np.array([0,0,0,eta,0,0,0,0])                
D4 = np.array([0,0,0,0,eta,0,0,0])                
D5 = np.array([0,0,0,0,0,eta,0,0])                
D6 = np.array([0,0,0,0,0,0,nu,0])                
D7 = np.array([0,0,0,0,2*eta,0,2*eta,nu])              
D = np.array([D0,D1,D2,D3,D4,D5,D6,D7])


# Eigenvalues (advection speed and characteristic diffusion) and eigenvectors (initial conditions)
v,w = LA.eig(A)
indicespos = np.array([])
indicesneg = np.array([])
d,e = LA.eig(D)
d = np.max(d)


# Sort by positve and negative advection speeds
for i in range(0,v.size):
    if v[i]>0:
        indicespos = np.append(indicespos,i)
    elif v[i]<0:
        indicesneg = np.append(indicesneg,i)
        
        
        
        
# Choice of initial condition        
print('Positive advection speed:'+str(indicespos))
print('Negative advection speed:'+str(indicesneg))
initial = int(input("Choose your initial condition ")) 




# Normalisation of inital condition
Uinit = a*w[:,initial]/np.amax(abs(w[:,initial]))
v = v[initial]









# Discretisation of space and time
deltax = 2*L/Nx
if method==0:
    deltat = CFL*(-d/v**2+deltax/v**2*np.sqrt(d**2/deltax**2+v**2))
elif method==1:
    deltat = deltax*CFL/np.abs(v)
Nt = np.int(T/deltat)
T = Nt*deltat
x = np.linspace(-L,L,Nx+1)
t = np.linspace(0,T,Nt+1)
print('Number of time steps: '+str(Nt))




# Initial conditions with order (u1x,u1y,u1z,B1x,B1y,B1z,rho1,p1)
def initial_Conditions(x,fun):
    if fun==1:
        return np.outer(Uinit,np.sin(x*np.pi/L));
    elif fun==2:
        return np.outer(Uinit,1/np.cosh(x*40/L));






# Compute Flux F(U) and Jacobian J(U)=dF/dU
U0,U1,U2,U3,U4,U5,U6,U7 = symbols('U0 U1 U2 U3 U4 U5 U6 U7')
     
F0 = U1
F1 = U1**2/U0+(gamma-1)*(U7-1/(2*U0)*(U1**2+U2**2+U3**2)-1/2*(U4**2+U5**2+U6**2))+U5**2+U6**2
F2 = U1*U2/U0-U4*U5
F3 = U1*U3/U0-U4*U6
F4 = 0
F5 = U1*U5/U0-U2*U4/U0
F6 = U1*U6/U0-U3*U4/U0
F7 = U1**2/(2*U0**2)*(U1**2+U2**2+U3**2)+gamma*U1*U7/U0-gamma*U1/(2*U0**2)*(U1**2+U2**2+U3**2)-gamma*U1/(2*U0)*(U4**2+U5**2+U6**2)+U1*U6**2/U0-U3*U4*U6/U0-U2*U4*U5/U0+U1*U5**2/U0
F = Matrix([F0,F1,F2,F3,F4,F5,F6,F7])    
    
Flux = lambdify((U0,U1,U2,U3,U4,U5,U6,U7),F) 
U = Matrix([U0,U1,U2,U3,U4,U5,U6,U7])
J = F.jacobian(U)
Jacobian = lambdify((U0,U1,U2,U3,U4,U5,U6,U7),J)



# Conversion of Flux F(U) and Jacobian J(U)=dF/dU into callable functions
def FJ(U,flux,jacobian):
    F = flux(U[0],U[1],U[2],U[3],U[4],U[5],U[6],U[7])
    J = jacobian(U[0],U[1],U[2],U[3],U[4],U[5],U[6],U[7])
    return F.flatten(),J

    
    
# Compute diffusion D(U)
def D(U):
    diff = np.zeros(8)
    diff[0] = nu*U[0]
    diff[1] = nu*U[1]
    diff[2] = nu*U[2]
    diff[3] = nu*U[3]
    diff[4] = eta*U[4]
    diff[5] = eta*U[5]
    diff[6] = eta*U[6]
    diff[7] = eta*(U[4]**2+U[6]**2)+nu*U[7]
    return diff




# Lax-Wendroff-FTCS (Forward time and centered space) scheme for nonlinear MHD equations in one dimensions
def LaxWendroffFCTS():
    U = np.zeros((Nt+1,Nx+1,8))
    IC = initial_Conditions(x,init)
    U[0,:,0] = IC[6,:]+rho0                                                          
    U[0,:,1] = (IC[6,:]+rho0)*IC[0,:]
    U[0,:,2] = (IC[6,:]+rho0)*IC[1,:]
    U[0,:,3] = (IC[6,:]+rho0)*IC[2,:]
    U[0,:,4] = IC[3,:]+B0x
    U[0,:,5] = IC[4,:]+B0y
    U[0,:,6] = IC[5,:]+B0z
    U[0,:,7] = 1/2*(IC[6,:]+rho0)*(IC[0,:]**2+IC[1,:]**2+IC[2,:]**2)+(IC[7,:]+p0)/(2*(gamma-1))+1/2*((IC[3,:]+B0x)**2+(IC[4,:]+B0y)**2+(IC[5,:]+B0z)**2)                                     
    for n in range(0,Nt):
        if n%100==0:
            print('Time steps finished: '+str(n))
        for j in range(0,Nx):
            if j==0:
                U[n+1,j,:] = U[n,j,:]-deltat/(2*deltax)*(FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,-2,:],Flux,Jacobian)[0])+deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,j+1,:]),Flux,Jacobian)[1],FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,j,:],Flux,Jacobian)[0]))-deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,-2,:]),Flux,Jacobian)[1],FJ(U[n,j,:],Flux,Jacobian)[0]-FJ(U[n,-2,:],Flux,Jacobian)[0]))+deltat/deltax**2*(D(U[n,-2,:])-2*D(U[n,j,:])+D(U[n,j+1,:]))
                U[n+1,-1,:] = U[n+1,0,:]
            elif j==Nx-1:
                U[n+1,j,:] = U[n,j,:]-deltat/(2*deltax)*(FJ(U[n,0,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0])+deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,0,:]),Flux,Jacobian)[1],FJ(U[n,0,:],Flux,Jacobian)[0]-FJ(U[n,j,:],Flux,Jacobian)[0]))-deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,j-1,:]),Flux,Jacobian)[1],FJ(U[n,j,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0]))+deltat/deltax**2*(D(U[n,j-1,:])-2*D(U[n,j,:])+D(U[n,0,:]))
                U[n+1,-1,:] = U[n+1,0,:]
            else:
                U[n+1,j,:] = U[n,j,:]-deltat/(2*deltax)*(FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0])+deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,j+1,:]),Flux,Jacobian)[1],FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,j,:],Flux,Jacobian)[0]))-deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,j-1,:]),Flux,Jacobian)[1],FJ(U[n,j,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0]))+deltat/deltax**2*(D(U[n,j-1,:])-2*D(U[n,j,:])+D(U[n,j+1,:]))
                U[n+1,-1,:] = U[n+1,0,:]
    
    P = np.zeros((Nt+1,Nx+1,8))
    P[:,:,0] = U[:,:,0]
    P[:,:,1] = U[:,:,1]/U[:,:,0]
    P[:,:,2] = U[:,:,2]/U[:,:,0]
    P[:,:,3] = U[:,:,3]/U[:,:,0]
    P[:,:,4] = U[:,:,4]
    P[:,:,5] = U[:,:,5]
    P[:,:,6] = U[:,:,6]
    P[:,:,7] = (gamma-1)*(2*U[:,:,7]-1/U[:,:,0]*(U[0,:,1]**2+U[:,:,2]**2+U[:,:,3]**2)-(U[:,:,4]**2+U[:,:,5]**2+U[:,:,6]**2))
    print('Time integration finished!')
    return P








# Leapfrog-DuFort-Frankel scheme for nonlinear MHD equations in one dimension
def LeapfrogDufortFrankel():
    U = np.zeros((Nt+1,Nx+1,8))
    IC = initial_Conditions(x,init)
    alphanu = 1+2*deltat*nu/deltax**2
    alphaeta = 1+2*deltat*eta/deltax**2
    U[0,:,0] = IC[6,:]+rho0                                                          
    U[0,:,1] = (IC[6,:]+rho0)*IC[0,:]
    U[0,:,2] = (IC[6,:]+rho0)*IC[1,:]
    U[0,:,3] = (IC[6,:]+rho0)*IC[2,:]
    U[0,:,4] = IC[3,:]+B0x
    U[0,:,5] = IC[4,:]+B0y
    U[0,:,6] = IC[5,:]+B0z
    U[0,:,7] = 1/2*(IC[6,:]+rho0)*(IC[0,:]**2+IC[1,:]**2+IC[2,:]**2)+(IC[7,:]+p0)/(2*(gamma-1))+1/2*((IC[3,:]+B0x)**2+(IC[4,:]+B0y)**2+(IC[5,:]+B0z)**2)                                       
    for n in range(0,Nt):
        if n%100==0:
            print('Time steps finished: '+str(n))
        if n==0:
            for j in range(0,Nx):
                if j==0:
                    U[n+1,j,:] = 0.5*(U[n,j+1,:]+U[n,-2,:])-deltat/(2*deltax)*(FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,-2,:],Flux,Jacobian)[0])
                    U[n+1,-1,:] = U[n+1,0,:]
                elif j==Nx-1:
                    U[n+1,j,:] = 0.5*(U[n,0,:]+U[n,j-1,:])-deltat/(2*deltax)*(FJ(U[n,0,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0])
                    U[n+1,-1,:] = U[n+1,0,:]
                else:
                    U[n+1,j,:] = 0.5*(U[n,j+1,:]+U[n,j-1,:])-deltat/(2*deltax)*(FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0])
                    U[n+1,-1,:] = U[n+1,0,:]
        
        
        else:    
            for j in range(0,Nx):
                if j==0:
                    # Density integration
                    U[n+1,j,0] = U[n-1,j,0]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][0]-FJ(U[n,-2,:],Flux,Jacobian)[0][0])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,0]+U[n,-2,0]-U[n-1,j,0])
                    U[n+1,-1,0] = U[n+1,0,0]
                    
                    # Momentum integration
                    U[n+1,j,1] = U[n-1,j,1]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][1]-FJ(U[n,-2,:],Flux,Jacobian)[0][1])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,1]+U[n,-2,1]-U[n-1,j,1])
                    U[n+1,-1,1] = U[n+1,0,1]
                    U[n+1,j,2] = U[n-1,j,2]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][2]-FJ(U[n,-2,:],Flux,Jacobian)[0][2])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,2]+U[n,-2,2]-U[n-1,j,2])
                    U[n+1,-1,2] = U[n+1,0,2]
                    U[n+1,j,3] = U[n-1,j,3]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][3]-FJ(U[n,-2,:],Flux,Jacobian)[0][3])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,3]+U[n,-2,3]-U[n-1,j,3])
                    U[n+1,-1,3] = U[n+1,0,3]
                    
                    # Magnetic field integration
                    U[n+1,j,4] = U[n-1,j,4]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][4]-FJ(U[n,-2,:],Flux,Jacobian)[0][4])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,j+1,4]+U[n,-2,4]-U[n-1,j,4])
                    U[n+1,-1,4] = U[n+1,0,4]
                    U[n+1,j,5] = U[n-1,j,5]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][5]-FJ(U[n,-2,:],Flux,Jacobian)[0][5])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,j+1,5]+U[n,-2,5]-U[n-1,j,5])
                    U[n+1,-1,5] = U[n+1,0,5]
                    U[n+1,j,6] = U[n-1,j,6]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][6]-FJ(U[n,-2,:],Flux,Jacobian)[0][6])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,j+1,4]+U[n,-2,6]-U[n-1,j,6])
                    U[n+1,-1,6] = U[n+1,0,6]
                    
                    # Energy integration
                    U[n+1,j,7] = U[n-1,j,7]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][7]-FJ(U[n,j-1,:],Flux,Jacobian)[0][7])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,7]+U[n,-2,7]-U[n-1,j,7])+2*deltat*eta/(alphanu*deltax**2)*(U[n,j+1,4]**2+U[n,j+1,6]**2+U[n,-2,4]**2+U[n,-2,6]**2-U[n+1,j,4]**2-U[n+1,j,6]**2-U[n-1,j,4]**2-U[n-1,j,6]**2)
                    U[n+1,-1,7] = U[n+1,0,7]
                elif j==Nx-1:
                    # Density integration
                    U[n+1,j,0] = U[n-1,j,0]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,0,:],Flux,Jacobian)[0][0]-FJ(U[n,j-1,:],Flux,Jacobian)[0][0])+2*deltat*nu/(alphanu*deltax**2)*(U[n,0,0]+U[n,j-1,0]-U[n-1,j,0])
                    U[n+1,-1,0] = U[n+1,0,0]
                    
                    # Momentum integration
                    U[n+1,j,1] = U[n-1,j,1]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,0,:],Flux,Jacobian)[0][1]-FJ(U[n,j-1,:],Flux,Jacobian)[0][1])+2*deltat*nu/(alphanu*deltax**2)*(U[n,0,1]+U[n,j-1,1]-U[n-1,j,1])
                    U[n+1,-1,1] = U[n+1,0,1]
                    U[n+1,j,2] = U[n-1,j,2]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,0,:],Flux,Jacobian)[0][2]-FJ(U[n,j-1,:],Flux,Jacobian)[0][2])+2*deltat*nu/(alphanu*deltax**2)*(U[n,0,2]+U[n,j-1,2]-U[n-1,j,2])
                    U[n+1,-1,2] = U[n+1,0,2]
                    U[n+1,j,3] = U[n-1,j,3]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,0,:],Flux,Jacobian)[0][3]-FJ(U[n,j-1,:],Flux,Jacobian)[0][3])+2*deltat*nu/(alphanu*deltax**2)*(U[n,0,3]+U[n,j-1,3]-U[n-1,j,3])
                    U[n+1,-1,3] = U[n+1,0,3]
                    
                    # Magnetic field integration
                    U[n+1,j,4] = U[n-1,j,4]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,0,:],Flux,Jacobian)[0][4]-FJ(U[n,j-1,:],Flux,Jacobian)[0][4])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,0,4]+U[n,j-1,4]-U[n-1,j,4])
                    U[n+1,-1,4] = U[n+1,0,4]
                    U[n+1,j,5] = U[n-1,j,5]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,0,:],Flux,Jacobian)[0][5]-FJ(U[n,j-1,:],Flux,Jacobian)[0][5])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,0,5]+U[n,j-1,5]-U[n-1,j,5])
                    U[n+1,-1,5] = U[n+1,0,5]
                    U[n+1,j,6] = U[n-1,j,6]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,0,:],Flux,Jacobian)[0][6]-FJ(U[n,j-1,:],Flux,Jacobian)[0][6])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,0,4]+U[n,j-1,6]-U[n-1,j,6])
                    U[n+1,-1,6] = U[n+1,0,6]
                    
                    # Energy integration
                    U[n+1,j,7] = U[n-1,j,7]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,0,:],Flux,Jacobian)[0][7]-FJ(U[n,j-1,:],Flux,Jacobian)[0][7])+2*deltat*nu/(alphanu*deltax**2)*(U[n,0,7]+U[n,j-1,7]-U[n-1,j,7])+2*deltat*eta/(alphanu*deltax**2)*(U[n,0,4]**2+U[n,0,6]**2+U[n,j-1,4]**2+U[n,j-1,6]**2-U[n+1,j,4]**2-U[n+1,j,6]**2-U[n-1,j,4]**2-U[n-1,j,6]**2)
                    U[n+1,-1,7] = U[n+1,0,7]
                else:
                    # Density integration
                    U[n+1,j,0] = U[n-1,j,0]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][0]-FJ(U[n,j-1,:],Flux,Jacobian)[0][0])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,0]+U[n,j-1,0]-U[n-1,j,0])
                    U[n+1,-1,0] = U[n+1,0,0]
                    
                    # Momentum integration
                    U[n+1,j,1] = U[n-1,j,1]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][1]-FJ(U[n,j-1,:],Flux,Jacobian)[0][1])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,1]+U[n,j-1,1]-U[n-1,j,1])
                    U[n+1,-1,1] = U[n+1,0,1]
                    U[n+1,j,2] = U[n-1,j,2]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][2]-FJ(U[n,j-1,:],Flux,Jacobian)[0][2])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,2]+U[n,j-1,2]-U[n-1,j,2])
                    U[n+1,-1,2] = U[n+1,0,2]
                    U[n+1,j,3] = U[n-1,j,3]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][3]-FJ(U[n,j-1,:],Flux,Jacobian)[0][3])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,3]+U[n,j-1,3]-U[n-1,j,3])
                    U[n+1,-1,3] = U[n+1,0,3]
                    
                    # Magnetic field integration
                    U[n+1,j,4] = U[n-1,j,4]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][4]-FJ(U[n,j-1,:],Flux,Jacobian)[0][4])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,j+1,4]+U[n,j-1,4]-U[n-1,j,4])
                    U[n+1,-1,4] = U[n+1,0,4]
                    U[n+1,j,5] = U[n-1,j,5]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][5]-FJ(U[n,j-1,:],Flux,Jacobian)[0][5])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,j+1,5]+U[n,j-1,5]-U[n-1,j,5])
                    U[n+1,-1,5] = U[n+1,0,5]
                    U[n+1,j,6] = U[n-1,j,6]/alphaeta-deltat/(deltax*alphaeta)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][6]-FJ(U[n,j-1,:],Flux,Jacobian)[0][6])+2*deltat*eta/(alphaeta*deltax**2)*(U[n,j+1,4]+U[n,j-1,6]-U[n-1,j,6])
                    U[n+1,-1,6] = U[n+1,0,6]
                    
                    # Energy integration
                    U[n+1,j,7] = U[n-1,j,7]/alphanu-deltat/(deltax*alphanu)*(FJ(U[n,j+1,:],Flux,Jacobian)[0][7]-FJ(U[n,j-1,:],Flux,Jacobian)[0][7])+2*deltat*nu/(alphanu*deltax**2)*(U[n,j+1,7]+U[n,j-1,7]-U[n-1,j,7])+2*deltat*eta/(alphanu*deltax**2)*(U[n,j+1,4]**2+U[n,j+1,6]**2+U[n,j-1,4]**2+U[n,j-1,6]**2-U[n+1,j,4]**2-U[n+1,j,6]**2-U[n-1,j,4]**2-U[n-1,j,6]**2)
                    U[n+1,-1,7] = U[n+1,0,7]
                    
    P = np.zeros((Nt+1,Nx+1,8))
    P[:,:,0] = U[:,:,0]
    P[:,:,1] = U[:,:,1]/U[:,:,0]
    P[:,:,2] = U[:,:,2]/U[:,:,0]
    P[:,:,3] = U[:,:,3]/U[:,:,0]
    P[:,:,4] = U[:,:,4]
    P[:,:,5] = U[:,:,5]
    P[:,:,6] = U[:,:,6]
    P[:,:,7] = (gamma-1)*(2*U[:,:,7]-1/U[:,:,0]*(U[0,:,1]**2+U[:,:,2]**2+U[:,:,3]**2)-(U[:,:,4]**2+U[:,:,5]**2+U[:,:,6]**2))          
    print('Time integration finished!')
    return P






if method==0:
    Test = LaxWendroffFCTS()
elif method==1:    
    Test = LeapfrogDufortFrankel()









# Animation 
print('Start to prepare animation')
f, (a1, a2, a3, a4, a5, a6, a7, a8) = plt.subplots(8,1,sharex=True)
f.set_figheight(13)
f.set_figwidth(7)
a1.set_title('t = 0.00')
a1.set_xlim((-L,L))
a2.set_xlim((-L,L))
a3.set_xlim((-L,L))
a4.set_xlim((-L,L))
a5.set_xlim((-L,L))
a6.set_xlim((-L,L))
a7.set_xlim((-L,L))
a8.set_xlim((-L,L))
a1.set_ylim((np.min(Test[:,:,0])-0.0001,np.max(Test[:,:,0])+0.0001))
a2.set_ylim((np.min(Test[:,:,1])-0.0001,np.max(Test[:,:,1])+0.0001))
a3.set_ylim((np.min(Test[:,:,2])-0.0001,np.max(Test[:,:,2])+0.0001))
a4.set_ylim((np.min(Test[:,:,3])-0.0001,np.max(Test[:,:,3])+0.0001))
a5.set_ylim((np.min(Test[:,:,4])-0.0001,np.max(Test[:,:,4])+0.0001))
a6.set_ylim((np.min(Test[:,:,5])-0.0001,np.max(Test[:,:,5])+0.0001))
a7.set_ylim((np.min(Test[:,:,6])-0.0001,np.max(Test[:,:,6])+0.0001))
a8.set_ylim((np.min(Test[:,:,7])-0.0001,np.max(Test[:,:,7])+0.0001))
a1.set_ylabel('rho')
a2.set_ylabel('ux')
a3.set_ylabel('uy')
a4.set_ylabel('uz')
a5.set_ylabel('Bx')
a6.set_ylabel('By')
a7.set_ylabel('Bz')
a8.set_ylabel('p')
a1.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
a2.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
a3.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
a4.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
a5.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
a6.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
a7.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off') 
line1,=a1.plot([],[], lw=2)
line2,=a2.plot([],[], lw=2)
line3,=a3.plot([],[], lw=2)
line4,=a4.plot([],[], lw=2)
line5,=a5.plot([],[], lw=2)
line6,=a6.plot([],[], lw=2)
line7,=a7.plot([],[], lw=2)
line8,=a8.plot([],[], lw=2)
a8.set_xlabel('x')
plt.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.1)

def init():  
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    line4.set_data([], [])
    line5.set_data([], [])
    line6.set_data([], [])
    line7.set_data([], [])
    line8.set_data([], [])
    return line1,

def animate(i):
    ti = int(fr*i)
    line1.set_data(x,Test[ti,:,0])
    line2.set_data(x,Test[ti,:,1])
    line3.set_data(x,Test[ti,:,2])
    line4.set_data(x,Test[ti,:,3])
    line5.set_data(x,Test[ti,:,4])
    line6.set_data(x,Test[ti,:,5])
    line7.set_data(x,Test[ti,:,6])
    line8.set_data(x,Test[ti,:,7])
    a1.set_title('t='+'%.2f' % t[ti])
    return line1,

animation.FuncAnimation(f, animate, init_func=init, frames=int(Nt/fr), interval=50, blit=True)

Positive advection speed:[1. 3. 5.]
Negative advection speed:[2. 4. 6.]
Choose your initial condition 1
Number of time steps: 4828
Time steps finished: 0
Time steps finished: 100
Time steps finished: 200


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-2c8b232ea83a>", line 342, in <module>
    Test = LaxWendroffFCTS()
  File "<ipython-input-1-2c8b232ea83a>", line 197, in LaxWendroffFCTS
    U[n+1,j,:] = U[n,j,:]-deltat/(2*deltax)*(FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0])+deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,j+1,:]),Flux,Jacobian)[1],FJ(U[n,j+1,:],Flux,Jacobian)[0]-FJ(U[n,j,:],Flux,Jacobian)[0]))-deltat**2/(2*deltax**2)*(np.dot(FJ(0.5*(U[n,j,:]+U[n,j-1,:]),Flux,Jacobian)[1],FJ(U[n,j,:],Flux,Jacobian)[0]-FJ(U[n,j-1,:],Flux,Jacobian)[0]))+deltat/deltax**2*(D(U[n,j-1,:])-2*D(U[n,j,:])+D(U[n,j+1,:]))
  File "<ipython-input-1-2c8b232ea83a>", line 153, in FJ
    J = jacobian(U[0],U[1],U[2],U[3],U[4],U[5],U[6],U[7])
  File "<string>", line 1, in <lambda>
KeyboardInterrupt

During handlin

KeyboardInterrupt: 